This notebook is for figuring out how to work with the FMA dataset

In [31]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib
import sklearn
import os.path
import sklearn

In [32]:
features = pd.read_csv("fma_metadata/features.csv", index_col=0, header=[0,1,2])
genres = pd.read_csv("fma_metadata/genres.csv")
tracks = pd.read_csv("fma_metadata/tracks.csv", index_col=0, header=[0,1])

In [33]:
small = tracks['set','subset'] == 'small'
small_tracks = tracks[small]

In [34]:
all_genres = small_tracks['track','genre_top']
for genre in all_genres:
    if genre not in small_tracks['track','genre_top'].unique():
        print('not in list')

In [35]:
small_features = features[small]

In [36]:
len(small_features['chroma_cens'].columns)

84

In [37]:
small_features.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
140           0.533579 -0.623885 -1.086205 -1.081079 -0.765151 -0.072282   
141           0.172898 -0.284804 -1.169662 -1.062855 -0.706868 -0.708281   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
140        -0.882913 -0.582376 -0.884749 -0.645214  ...  0.157683  0.028070   
141        -0.204884  0.023624 -0.642770 -0.786291  ...  0.145994  0.024342   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
140         0.025946  11.052547  0.379395  0.052379  0.036621  0.001953   
141         0.032111  32.994659  0.415527  0.040267  0.034668  0.002930   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
5           2.193303  0.044861  
10          3.542325  0.040800  
140         3.143968  0.057712  
141         4.204097  0.028665  

[5 rows x 518 columns]

In [38]:
small_features['mfcc'].head()

statistics  kurtosis                                                    \
number            01        02        03        04        05        06   
track_id                                                                 
2           3.856789  1.541901  0.000816  0.330728  0.118731 -0.342687   
5           2.624517  2.415293  0.440233 -0.782131 -0.771069 -0.724216   
10          5.076893  1.161854  2.095651  1.372743 -0.203574 -0.345354   
140         0.384906  1.203378  0.429393  0.639471 -0.080144 -0.440946   
141        -0.038922  4.771146 -0.378949 -0.309047 -0.661308 -0.531602   

statistics                                          ...        std            \
number            07        08        09        10  ...         11        12   
track_id                                            ...                        
2          -0.259252  0.146735  0.410656 -0.162872  ...  10.059609  8.601942   
5           0.090260  0.152119  0.261731 -0.608905  ...   8.863638  9.581952   
10         -0.529139  0.561974  0.281350 -0.150672  ...   8.289734  7.985110   
140        -0.432567 -0.126163  0.507695  0.408915  ...   9.883532  8.250456   
141         0.029186  0.662524 -0.084562  0.965640  ...   9.464321  8.119837   

statistics                                                              \
number            13        14        15        16        17        18   
track_id                                                                 
2           9.284250  9.245516  8.520863  8.560472  7.651871  7.246555   
5           8.895723  8.141456  8.201844  7.780963  7.132692  7.539753   
10          7.075400  6.972649  7.071393  7.270959  7.051070  6.928591   
140         8.085891  8.114858  8.832722  8.157245  7.297486  7.734456   
141         8.918522  8.059117  9.339061  8.451561  8.426965  8.341757   

statistics                      
number            19        20  
track_id                        
2           7.077188  7.391859  
5           8.452527  7.334442  
10          6.430473  6.186294  
140         7.807103  7.986255  
141         8.572934  9.351485  

[5 rows x 140 columns]

In [39]:
len(tracks)

106574

In [40]:
len(small_tracks)

8000

In [41]:
small_tracks['track','genre_top'].unique() # All genres

array(['Hip-Hop', 'Pop', 'Folk', 'Experimental', 'Rock', 'International',
       'Electronic', 'Instrumental'], dtype=object)

In [42]:
for genre in small_tracks['track','genre_top'].unique():
    print(genre, len(small_tracks[small_tracks['track','genre_top'] == genre]))

Hip-Hop 1000
Pop 1000
Folk 1000
Experimental 1000
Rock 1000
International 1000
Electronic 1000
Instrumental 1000


Tracks are therefore evenly distributed in this dataset (8 genres, 1000 tracks each)

In [43]:
small_train = small_tracks[small_tracks['set','split'] == 'training']
small_val = small_tracks[small_tracks['set','split'] == 'validation']
small_test = small_tracks[small_tracks['set','split'] == 'test']

In [44]:
print(len(small_train))
print(len(small_val))
print(len(small_test))

6400
800
800


In [45]:
X_train = small_features.loc[small_tracks['set','split'] == 'training']
X_val = small_features.loc[small_tracks['set','split'] == 'validation']
X_test = small_features.loc[small_tracks['set','split'] == 'test']

In [46]:
y_train = small_train['track','genre_top']
y_val = small_val['track','genre_top']
y_test = small_test['track','genre_top']

In [47]:
print(len(y_train))
print(len(y_val))
print(len(y_test))

6400
800
800


In [48]:
le = sklearn.preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

In [49]:
le.classes_

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype=object)

In [50]:
y_train[:10]

array([3, 3, 6, 2, 2, 2, 2, 2, 2, 2])

In [51]:
small_train['track','genre_top'][:10]

track_id
2      Hip-Hop
5      Hip-Hop
10         Pop
140       Folk
141       Folk
190       Folk
193       Folk
194       Folk
197       Folk
200       Folk
Name: (track, genre_top), dtype: object

In [52]:
len(list(X_train['mfcc'].columns))

140

In [53]:
len(list(X_train['chroma_cens'].columns))

84

In [54]:
X_train_mfcc = X_train['mfcc']
X_val_mfcc = X_val['mfcc']
X_test_mfcc = X_test['mfcc']

In [55]:
X_train_chroma = X_train['chroma_cens']
X_val_chroma = X_val['chroma_cens']
X_test_chroma = X_test['chroma_cens']

In [56]:
genres = le.inverse_transform([0,1,2,3,4,5,6,7])
genres

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype=object)

In [57]:
C_params = [0.001, 0.1, 1, 5]
for c in C_params:
    print("C = ", c)
    lr = sklearn.linear_model.LogisticRegression(C=c, penalty='l2')
    lr.fit(X_train_mfcc, y_train)
    y_pred = lr.predict(X_test_mfcc)
    print(sklearn.metrics.classification_report(y_test, y_pred, target_names=genres))

C =  0.001


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.47      0.54      0.50       100
 Experimental       0.34      0.26      0.29       100
         Folk       0.19      0.19      0.19       100
      Hip-Hop       0.58      0.66      0.62       100
 Instrumental       0.47      0.43      0.45       100
International       0.42      0.47      0.44       100
          Pop       0.27      0.22      0.24       100
         Rock       0.58      0.61      0.60       100

    micro avg       0.42      0.42      0.42       800
    macro avg       0.41      0.42      0.42       800
 weighted avg       0.41      0.42      0.42       800

C =  0.1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.50      0.56      0.53       100
 Experimental       0.32      0.26      0.29       100
         Folk       0.17      0.15      0.16       100
      Hip-Hop       0.60      0.68      0.64       100
 Instrumental       0.49      0.45      0.47       100
International       0.39      0.43      0.41       100
          Pop       0.24      0.22      0.23       100
         Rock       0.58      0.63      0.60       100

    micro avg       0.42      0.42      0.42       800
    macro avg       0.41      0.42      0.41       800
 weighted avg       0.41      0.42      0.41       800

C =  1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.49      0.54      0.51       100
 Experimental       0.31      0.25      0.28       100
         Folk       0.16      0.14      0.15       100
      Hip-Hop       0.60      0.71      0.65       100
 Instrumental       0.52      0.46      0.49       100
International       0.39      0.42      0.40       100
          Pop       0.22      0.21      0.22       100
         Rock       0.57      0.64      0.60       100

    micro avg       0.42      0.42      0.42       800
    macro avg       0.41      0.42      0.41       800
 weighted avg       0.41      0.42      0.41       800

C =  5


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.48      0.54      0.51       100
 Experimental       0.30      0.24      0.27       100
         Folk       0.16      0.14      0.15       100
      Hip-Hop       0.60      0.70      0.65       100
 Instrumental       0.51      0.45      0.48       100
International       0.39      0.43      0.41       100
          Pop       0.23      0.22      0.23       100
         Rock       0.57      0.64      0.60       100

    micro avg       0.42      0.42      0.42       800
    macro avg       0.41      0.42      0.41       800
 weighted avg       0.41      0.42      0.41       800



In [58]:
C_params = [0.001, 0.1, 1, 5]
for c in C_params:
    print("C = ", c)
    lr = sklearn.linear_model.LogisticRegression(C=c, penalty='l2')
    lr.fit(X_train_chroma, y_train)
    y_pred = lr.predict(X_test_chroma)
    print(sklearn.metrics.classification_report(y_test, y_pred, target_names=genres))

C =  0.001


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.17      0.02      0.04       100
 Experimental       0.16      0.27      0.20       100
         Folk       0.10      0.18      0.13       100
      Hip-Hop       0.23      0.14      0.17       100
 Instrumental       0.18      0.33      0.23       100
International       0.09      0.05      0.06       100
          Pop       0.06      0.05      0.06       100
         Rock       0.24      0.14      0.18       100

    micro avg       0.15      0.15      0.15       800
    macro avg       0.15      0.15      0.13       800
 weighted avg       0.15      0.15      0.13       800

C =  0.1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.10      0.05      0.07       100
 Experimental       0.17      0.19      0.18       100
         Folk       0.09      0.12      0.10       100
      Hip-Hop       0.34      0.34      0.34       100
 Instrumental       0.23      0.30      0.26       100
International       0.21      0.18      0.19       100
          Pop       0.10      0.04      0.06       100
         Rock       0.21      0.31      0.25       100

    micro avg       0.19      0.19      0.19       800
    macro avg       0.18      0.19      0.18       800
 weighted avg       0.18      0.19      0.18       800

C =  1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.22      0.14      0.17       100
 Experimental       0.21      0.18      0.20       100
         Folk       0.17      0.21      0.19       100
      Hip-Hop       0.29      0.31      0.30       100
 Instrumental       0.26      0.34      0.30       100
International       0.27      0.30      0.28       100
          Pop       0.19      0.07      0.10       100
         Rock       0.24      0.34      0.28       100

    micro avg       0.24      0.24      0.24       800
    macro avg       0.23      0.24      0.23       800
 weighted avg       0.23      0.24      0.23       800

C =  5


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


               precision    recall  f1-score   support

   Electronic       0.21      0.13      0.16       100
 Experimental       0.24      0.18      0.20       100
         Folk       0.18      0.23      0.20       100
      Hip-Hop       0.27      0.31      0.29       100
 Instrumental       0.26      0.36      0.30       100
International       0.29      0.36      0.32       100
          Pop       0.23      0.08      0.12       100
         Rock       0.25      0.31      0.27       100

    micro avg       0.24      0.24      0.24       800
    macro avg       0.24      0.24      0.23       800
 weighted avg       0.24      0.24      0.23       800



In [59]:
for n in range(1,11):
    print("n_neighbors = ", n)
    neigh = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train_mfcc, y_train)
    y_pred = neigh.predict(X_test_mfcc)
    print(sklearn.metrics.classification_report(y_test, y_pred, target_names=genres))

n_neighbors =  1
               precision    recall  f1-score   support

   Electronic       0.38      0.33      0.35       100
 Experimental       0.13      0.12      0.12       100
         Folk       0.16      0.16      0.16       100
      Hip-Hop       0.47      0.58      0.52       100
 Instrumental       0.28      0.25      0.27       100
International       0.31      0.36      0.33       100
          Pop       0.22      0.21      0.22       100
         Rock       0.38      0.38      0.38       100

    micro avg       0.30      0.30      0.30       800
    macro avg       0.29      0.30      0.29       800
 weighted avg       0.29      0.30      0.29       800

n_neighbors =  2
               precision    recall  f1-score   support

   Electronic       0.32      0.50      0.39       100
 Experimental       0.18      0.25      0.21       100
         Folk       0.17      0.22      0.19       100
      Hip-Hop       0.46      0.56      0.51       100
 Instrumental       0.26   

In [60]:
for n in range(1,11):
    print("n_neighbors = ", n)
    neigh = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train_chroma, y_train)
    y_pred = neigh.predict(X_test_chroma)
    print(sklearn.metrics.classification_report(y_test, y_pred, target_names=genres))

n_neighbors =  1
               precision    recall  f1-score   support

   Electronic       0.15      0.15      0.15       100
 Experimental       0.16      0.20      0.18       100
         Folk       0.18      0.17      0.18       100
      Hip-Hop       0.17      0.14      0.15       100
 Instrumental       0.23      0.16      0.19       100
International       0.17      0.22      0.19       100
          Pop       0.13      0.14      0.14       100
         Rock       0.21      0.19      0.20       100

    micro avg       0.17      0.17      0.17       800
    macro avg       0.17      0.17      0.17       800
 weighted avg       0.17      0.17      0.17       800

n_neighbors =  2
               precision    recall  f1-score   support

   Electronic       0.16      0.27      0.20       100
 Experimental       0.12      0.25      0.16       100
         Folk       0.19      0.24      0.21       100
      Hip-Hop       0.12      0.11      0.11       100
 Instrumental       0.24   